In [24]:
import numpy as np
import pandas as pd

from sklearn import preprocessing #if it says that this dosent work look at the python 3 in the top right.

In [25]:
ORIGINAL_TRAINING_FILE = '/Users/ethan/Desktop/Ethan/Python/ML/framework/input/titanic/train.csv'
ORIGINAL_TEST_FILE = '/Users/ethan/Desktop/Ethan/Python/ML/framework/input/titanic/test.csv'


In [26]:
df = pd.read_csv(ORIGINAL_TRAINING_FILE)
test_df = pd.read_csv(ORIGINAL_TEST_FILE)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [27]:
df['hasCabin']=df['Cabin'].apply(lambda x: 0 if x==0 else 1)
df['Family'] = df['SibSp'] + df['Parch']
df.dropna(subset=['Embarked'], inplace=True)
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

#repeating for the test data:
test_df['hasCabin']=test_df['Cabin'].apply(lambda x: 0 if x==0 else 1)
test_df['Family'] = test_df['SibSp'] + test_df['Parch']
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())


In [28]:
values_to_be_dropped = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
categorical_values = ['Pclass', 'Sex', 'Embarked']

test_df = test_df.drop(values_to_be_dropped, axis = 1)
df = df.drop(values_to_be_dropped, axis = 1)

In [29]:
#using the one hot encoder:

def encode_categories(cat_values, dataframe):
    ohe = preprocessing.OneHotEncoder(sparse=False)
    for cat in cat_values:
        ohe_category = ohe.fit_transform(dataframe[cat].values.reshape(-1, 1))
        for i in range(ohe_category.shape[1]):
            title = cat + f'_{i}'
            dataframe[title] = ohe_category[:, i] #this is all of the values in the i column of the table
    dataframe = dataframe.drop(cat_values, axis = 1)
    return dataframe
            
df = encode_categories(categorical_values, df)
test_df = encode_categories(categorical_values, test_df)

In [30]:
test_df

,PassengerId,Age,Fare,hasCabin,Family,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,892,34.50000,7.8292,1,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,893,47.00000,7.0000,1,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,894,62.00000,9.6875,1,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,895,27.00000,8.6625,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,896,22.00000,12.2875,1,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,30.27259,8.0500,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
414,1306,39.00000,108.9000,1,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
415,1307,38.50000,7.2500,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
416,1308,30.27259,8.0500,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [31]:
#repalce the train data with this:
df.to_csv('/Users/ethan/Desktop/Ethan/Python/ML/framework/notebooks/processed_train.csv', index=False)
test_df.to_csv('/Users/ethan/Desktop/Ethan/Python/ML/framework/notebooks/processed_test.csv', index=False)